In [20]:
import pandas as pd

df = pd.read_parquet('train.parquet')

ssubmission = pd.read_csv('sample_submission.csv')
test = pd.read_csv('test.csv')

In [21]:
sample = df.iloc[0]

In [10]:
sample['content']

'Новий огляд мапи DeepState від російського військового експерта, кухара путіна 2 розряду, спеціаліста по снарядному голоду та ректора музичної академії міноборони рф Євгєнія Пригожина. \nПригожин прогнозує, що невдовзі настане день звільнення Криму і день розпаду росії. Каже, що передумови цього вже створені. \n*Відео взяли з каналу \nФД\n. \n@informnapalm'

In [19]:
span = sample['trigger_words'][0]
sample['content'][span[0]:span[1]]

'від російського військового експерта'

In [27]:
targets = ssubmission.set_index('id').columns

In [28]:
for col in targets:
    df[col] = 0

In [40]:
from collections.abc import Iterable

In [43]:
import numpy as np
for ind, row in df.iterrows():
    if isinstance(row['techniques'], Iterable):
        for t in row['techniques']:
            df.loc[ind, t] = 1

In [47]:
y_true = df[targets].values

In [48]:
y_zero = np.zeros_like(y_true)
y_one = np.ones_like(y_true)

In [50]:
from sklearn.metrics import f1_score

f1_score(
        y_true, 
        y_zero, 
        average="macro"
    )

0.0

In [51]:
from sklearn.metrics import f1_score

f1_score(
        y_true, 
        y_one, 
        average="macro"
    )

0.21299793313102136

In [67]:
from copy import deepcopy

marginal_f_scores = []

for ind, c in enumerate(targets):
    marginal_score = round(f1_score(
        df[c], 
        y_one[:, ind], 
        average="binary"
    ), 4)
    class_balance = round(df[c].mean(), 4)

    ss = deepcopy(ssubmission)
    ss[c] = 1
    ss.to_csv(f'submissions/clf_probbing_{ind}_{c}_balance_{class_balance}_marginal_{marginal_score}.csv')

In [66]:
ssubmission

,id,straw_man,appeal_to_fear,fud,bandwagon,whataboutism,loaded_language,glittering_generalities,euphoria,cherry_picking,cliche
0,521cd2e8-dd9f-42c4-98ba-c0c8890ff1ba,0,0,0,0,0,0,0,0,0,0
1,9b2a61e4-d14e-4ff7-b304-e73d720319bf,0,0,0,0,0,0,0,0,0,0
2,f0f1c236-80a8-4d25-b30c-a420a39be632,0,0,0,0,0,0,0,0,0,0
3,31ea05ba-2c2b-4b84-aba7-f3cf6841b204,0,0,0,0,0,0,0,0,0,0
4,a79e13ec-6d9a-40b5-b54c-7f4f743a7525,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5730,e8e22b6d-0068-4afb-b606-4a1baa8a8d4c,0,0,0,0,0,0,0,0,0,0
5731,8b1d69b4-69ce-4e40-b4ba-dd2f370a8b6f,0,0,0,0,0,0,0,0,0,0
5732,c2246217-3358-4f61-bda8-e2ec21aed5b2,0,0,0,0,0,0,0,0,0,0
5733,45aa63c4-2248-4a0e-8f66-f3d23b6828ed,0,0,0,0,0,0,0,0,0,0


In [64]:
ssubmission.sum()

id                         521cd2e8-dd9f-42c4-98ba-c0c8890ff1ba9b2a61e4-d...
straw_man                                                                  0
appeal_to_fear                                                             0
fud                                                                        0
bandwagon                                                                  0
whataboutism                                                               0
loaded_language                                                            0
glittering_generalities                                                    0
euphoria                                                                   0
cherry_picking                                                             0
cliche                                                                     0
dtype: object

In [53]:
np.mean(marginal_f_scores)

0.21299793313102136

In [57]:
np.mean(marginal_f_scores)/10

0.021299793313102135

In [54]:
pd.Series(marginal_f_scores)

0    0.069697
1    0.145560
2    0.183028
3    0.078914
4    0.079397
5    0.680932
6    0.224390
7    0.215686
8    0.236271
9    0.216103
dtype: float64

In [58]:
pd.Series(marginal_f_scores)/10

0    0.006970
1    0.014556
2    0.018303
3    0.007891
4    0.007940
5    0.068093
6    0.022439
7    0.021569
8    0.023627
9    0.021610
dtype: float64

In [59]:
df[targets].mean()

straw_man                  0.036107
appeal_to_fear             0.078493
fud                        0.100733
bandwagon                  0.041078
whataboutism               0.041340
loaded_language            0.516222
glittering_generalities    0.126374
euphoria                   0.120879
cherry_picking             0.133961
cliche                     0.121141
dtype: float64

In [55]:
df

,id,content,lang,manipulative,techniques,trigger_words,straw_man,appeal_to_fear,fud,bandwagon,whataboutism,loaded_language,glittering_generalities,euphoria,cherry_picking,cliche
0,0bb0c7fa-101b-4583-a5f9-9d503339141c,Новий огляд мапи DeepState від російського вій...,uk,True,"[euphoria, loaded_language]","[[27, 63], [65, 88], [90, 183], [186, 308]]",0,0,0,0,0,1,0,1,0,0
1,7159f802-6f99-4e9d-97bd-6f565a4a0fae,Недавно 95 квартал жёстко поглумился над русск...,ru,True,"[loaded_language, cherry_picking]","[[0, 40], [123, 137], [180, 251], [253, 274]]",0,0,0,0,0,1,0,0,1,0
2,e6a427f1-211f-405f-bd8b-70798458d656,🤩\nТим часом йде евакуація Бєлгородського авто...,uk,True,"[loaded_language, euphoria]","[[55, 100]]",0,0,0,0,0,1,0,1,0,0
3,1647a352-4cd3-40f6-bfa1-d87d42e34eea,В Україні найближчим часом мають намір посилит...,uk,False,None,None,0,0,0,0,0,0,0,0,0,0
4,9c01de00-841f-4b50-9407-104e9ffb03bf,"Расчёты 122-мм САУ 2С1 ""Гвоздика"" 132-й бригад...",ru,True,[loaded_language],"[[114, 144]]",0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3817,0e5dd135-ef41-48d3-b274-faedf3a2126c,🤭\nросія ставить ППО на дахах адмінбудівель\nр...,uk,True,"[loaded_language, euphoria]","[[2, 7], [43, 75], [77, 143], [145, 178], [340...",0,0,0,0,0,1,0,1,0,0
3818,08e6772a-9793-4ec9-babd-2a9e0e8b31f9,"К слову, Бабий не просто «ларечник», а и челов...",ru,True,[loaded_language],"[[25, 35], [103, 128], [149, 166]]",0,0,0,0,0,1,0,0,0,0
3819,d7cfa984-46f2-450d-b4ec-28a0b5d93756,"Глава ФСБ Бортников ответил журналистам, почем...",ru,False,None,None,0,0,0,0,0,0,0,0,0,0
3820,4256b2b8-43bc-4d90-95c4-5fb25f1ab0e3,В ДНР завозили наркотики в бытовой технике\nОб...,ru,False,None,None,0,0,0,0,0,0,0,0,0,0


In [56]:
ssubmission

,id,straw_man,appeal_to_fear,fud,bandwagon,whataboutism,loaded_language,glittering_generalities,euphoria,cherry_picking,cliche
0,521cd2e8-dd9f-42c4-98ba-c0c8890ff1ba,0,0,0,0,0,0,0,0,0,0
1,9b2a61e4-d14e-4ff7-b304-e73d720319bf,0,0,0,0,0,0,0,0,0,0
2,f0f1c236-80a8-4d25-b30c-a420a39be632,0,0,0,0,0,0,0,0,0,0
3,31ea05ba-2c2b-4b84-aba7-f3cf6841b204,0,0,0,0,0,0,0,0,0,0
4,a79e13ec-6d9a-40b5-b54c-7f4f743a7525,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
5730,e8e22b6d-0068-4afb-b606-4a1baa8a8d4c,0,0,0,0,0,0,0,0,0,0
5731,8b1d69b4-69ce-4e40-b4ba-dd2f370a8b6f,0,0,0,0,0,0,0,0,0,0
5732,c2246217-3358-4f61-bda8-e2ec21aed5b2,0,0,0,0,0,0,0,0,0,0
5733,45aa63c4-2248-4a0e-8f66-f3d23b6828ed,0,0,0,0,0,0,0,0,0,0
